### Optimizing a portfolio when we know the future

This notebook aims to optimize a portfolio when we know the future. Essentially we are backfitting what would the best portfolio at a certain time, given that we know the future. 

Assumes the following,
- Convex
- Solvency II as a constraint

NB: Able to test different risk measures.

In [98]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [99]:
assets = ["Equity1", "Bond1"]
returns_df = pd.DataFrame({
    "Equity1": [0.01, 0.02, -0.01, 0.03, 0.02, 0.05, 0.01],  # Future returns (01/01/2014 to 01/07/2014)
    "Bond1": [0.005, 0.004, 0.002, 0.003, 0.0073, 0.092, 0.002]
}, index=pd.date_range("2014-01-01", periods=7, freq='W'))  # Weekly returns

returns_df

,Equity1,Bond1
2014-01-05,0.01,0.0050
2014-01-12,0.02,0.0040
2014-01-19,-0.01,0.0020
2014-01-26,0.03,0.0030
2014-02-02,0.02,0.0073
2014-02-09,0.05,0.0920
2014-02-16,0.01,0.0020


In [100]:
alpha = 0.95  # Confidence level for CVaR
n = len(assets)  # Number of assets
future_returns = returns_df.to_numpy()  # Actual future return data (e.g., 01/01/2014 to 01/01/2015)

In [101]:
# Optimization variables (allocations to each asset)
allocations = cp.Variable(n)
allocations

Variable((2,), var849)

In [102]:
# Risk Measures
def calculate_risk(allocations, returns, risk_measure="CVaR"):
    if risk_measure == "CVaR":
        # Calculate CVaR using the future returns
        VaR = cp.Variable()  # Value at Risk
        CVaR = cp.Variable()  # Conditional Value at Risk
        losses = -returns @ allocations
        VaR_constraint = losses >= VaR
        CVaR_expression = VaR + (1 / (1 - alpha)) * cp.sum(cp.pos(losses - VaR)) / len(returns)
        return CVaR_expression, [VaR_constraint]

    elif risk_measure == "variance":
        # Portfolio variance as a risk measure
        cov_matrix = np.cov(returns, rowvar=False)  # Covariance matrix based on returns
        portfolio_risk = cp.quad_form(allocations, cov_matrix)  # Variance (quadratic form)
        return portfolio_risk, []

    # Add more risk measures as needed (e.g., VaR, DVaR, etc.)
    else:
        raise ValueError("Unsupported risk measure")

In [103]:
# Calculate future returns from 01/01/2014 onward
risk_expression, risk_constraints = calculate_risk(allocations, future_returns, risk_measure="CVaR")

# Solvency II constraint: Ensure CVaR or another risk measure stays below a solvency threshold
SCR_threshold = 0.05  # Solvency Capital Requirement (set as a threshold for the risk measure)
solvency_constraint = risk_expression <= SCR_threshold

# Objective: Maximize expected return while minimizing risk (here CVaR, but it could be any risk measure)
expected_return = future_returns.mean(axis=0) @ allocations

print("Expected Return:", expected_return.value)
print("Risk Expression:", risk_expression.value)

print("Future Returns Shape:", future_returns.shape)
print("Allocations Shape:", allocations.shape)



Expected Return: None
Risk Expression: None
Future Returns Shape: (7, 2)
Allocations Shape: (2,)


In [104]:
# Set up the optimization problem with constraints
constraints = [
    cp.sum(allocations) == 1,  # Weights sum to 100%
    allocations >= 0           # No short-selling (non-negative weights)
] + risk_constraints + [solvency_constraint]

In [105]:
# Define the optimization problem (maximize expected return minus risk)
problem = cp.Problem(cp.Maximize(expected_return - risk_expression), constraints)


In [106]:
# Solve the problem
problem.solve(verbose = True)

# Get the optimal allocations
optimal_allocations = allocations.value
print("Optimal Allocations:", optimal_allocations)

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Oct 15 03:24:39 PM: Your problem has 3 variables, 11 constraints, and 0 parameters.
(CVXPY) Oct 15 03:24:39 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 15 03:24:39 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 15 03:24:39 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 15 03:24:39 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 15 03:24:39 PM: Compiling problem (target solver=CLARABEL).


In [107]:
print("Problem Status:", problem.status)


Problem Status: infeasible
